In [ ]:
#!/usr/bin/env python3
import gymnasium as gym

from collections import namedtuple
import numpy as np

from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim

In [ ]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
device = torch.device("cpu")
device

In [ ]:
HIDDEN_SIZE = 128
BATCH_SIZE = 100
PERCENTILE = 30
GAMMA = 0.9

In [ ]:
class DiscreteOneHotWrapper(gym.ObservationWrapper):
    def __init__(self, env):
        super(DiscreteOneHotWrapper, self).__init__(env)
        assert isinstance(env.observation_space,
                          gym.spaces.Discrete)
        shape = (env.observation_space.n, )
        self.observation_space = gym.spaces.Box(
            0.0, 1.0, shape, dtype=np.float32)

    def observation(self, observation):
        res = np.copy(self.observation_space.low)
        res[observation] = 1.0
        return res

In [ ]:
class MLP(nn.Module):
    def __init__(self, obs_size, hidden_size, n_actions):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, n_actions)
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
Episode = namedtuple('Episode', field_names=['reward', 'steps'])
EpisodeStep = namedtuple('EpisodeStep', field_names=['observation', 'action'])

In [ ]:
def iterate_batches(env, net, batch_size):
    batch = []
    episode_reward = 0.0
    episode_steps = []
    obs, _ = env.reset()
    sm = nn.Softmax(dim=1).to(device)
    while True:
        obs_v = torch.FloatTensor([obs]).to(device)
        act_probs_v = sm(net(obs_v))
        act_probs = act_probs_v.detach().cpu().numpy()[0]
        action = np.random.choice(len(act_probs), p=act_probs)
        next_obs, reward, is_done, truncated, _ = env.step(action)
        episode_reward += reward
        episode_steps.append(EpisodeStep(observation=obs, action=action))
        if is_done or truncated:
            batch.append(Episode(reward=episode_reward, steps=episode_steps))
            episode_reward = 0.0
            episode_steps = []
            next_obs, _ = env.reset()
            if len(batch) == batch_size:
                yield batch
                batch = []
        obs = next_obs

In [ ]:
def filter_batch(batch, percentile):
    filter_fun = lambda s: s.reward * (GAMMA ** len(s.steps))
    disc_rewards = list(map(filter_fun, batch))
    reward_bound = np.percentile(disc_rewards, percentile)

    train_obs = []
    train_act = []
    elite_batch = []
    for example, discounted_reward in zip(batch, disc_rewards):
        if discounted_reward > reward_bound:
            train_obs.extend(map(lambda step: step.observation,
                                 example.steps))
            train_act.extend(map(lambda step: step.action,
                                 example.steps))
            elite_batch.append(example)

    return elite_batch, train_obs, train_act, reward_bound

In [ ]:
from gymnasium.envs.toy_text.frozen_lake import generate_random_map

env = DiscreteOneHotWrapper(gym.make("FrozenLake-v1", desc=generate_random_map(size=4), is_slippery=False))

# env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

net = MLP(obs_size, 64, n_actions).to(device)

objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
writer = SummaryWriter(comment="-frozenlake-mlp")

In [ ]:
'number of parameters', sum(p.numel() for p in net.parameters())

In [ ]:
full_batch = []

with torch.device(device):
    for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
        
        reward_mean = float(np.mean(list(map(lambda s: s.reward, batch))))

        full_batch, obs, acts, reward_bound = filter_batch(full_batch + batch, PERCENTILE)

        if not full_batch:
            continue

        obs_v = torch.FloatTensor(obs).to(device)
        acts_v = torch.LongTensor(acts).to(device)
        full_batch = full_batch[-500:]

        optimizer.zero_grad()

        action_scores_v = net(obs_v)

        loss_v = objective(action_scores_v, acts_v)

        loss_v.backward()
        optimizer.step()

        print("%d: loss=%.3f, rw_mean=%.3f, rw_bound=%.3f, batch=%d" % (
            iter_no, loss_v.item(), reward_mean, reward_bound, len(full_batch))
        )

        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_mean", reward_mean, iter_no)
        writer.add_scalar("reward_bound", reward_bound, iter_no)

        if reward_mean > 0.8:
            print("Solved!")
            break

    writer.close()

In [ ]:
%load_ext autoreload
%autoreload 2

from collections import OrderedDict
from typing import Literal
from lib.architecture import RandomizedSearch, Search
from lib.layers import ExpandLinear
from lib.sample import SampleNormal, SampleUniform

def create_randomized_model(input_size, hidden_size, output_size, distribution: Literal['normal', 'uniform'] = 'normal'):
    encoder = nn.Sequential(
        nn.Linear(input_size, hidden_size),
        nn.LayerNorm(hidden_size)
    )

    search = RandomizedSearch(
        transition=nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.GELU(),
            nn.Linear(hidden_size, 2*hidden_size),
        ),
        sample=nn.Sequential(
            SampleNormal(hidden_size, num_samples=4) if distribution == 'normal' else SampleUniform(hidden_size, num_samples=4),
            nn.LayerNorm(hidden_size)
        ),
        fitness=nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.GELU(),
            nn.Linear(hidden_size, 1),
        ),
        max_depth=0,
        beam_width=4,
        reuse_parents=False
    )

    decoder = nn.Sequential(
        nn.Linear(hidden_size, output_size)
    )

    model = nn.Sequential(OrderedDict([
        ('encoder', encoder),
        ('search', search),
        ('decoder', decoder)
    ]))

    return model

def create_deterministic_model(input_size, hidden_size, output_size):
    encoder = nn.Sequential(
        nn.Linear(input_size, hidden_size),
        nn.LayerNorm(hidden_size)
    )

    search = Search(
        transition=nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            ExpandLinear(hidden_size, 4),
            nn.LayerNorm(hidden_size)
        ),
        fitness=nn.Sequential(
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1),
        ),
        max_depth=2,
        beam_width=4
    )

    decoder = nn.Sequential(
        nn.Linear(hidden_size, output_size)
    )

    model = nn.Sequential(OrderedDict([
        ('encoder', encoder),
        ('search', search),
        ('decoder', decoder)
    ]))

    return model

In [ ]:
from gymnasium.envs.toy_text.frozen_lake import generate_random_map

env = DiscreteOneHotWrapper(gym.make("FrozenLake-v1", desc=generate_random_map(size=4), is_slippery=False))

# env = gym.wrappers.Monitor(env, directory="mon", force=True)
obs_size = env.observation_space.shape[0]
n_actions = env.action_space.n

net = create_randomized_model(obs_size, 16, n_actions, distribution='uniform').to(device)
# net = create_deterministic_model(obs_size, 16, n_actions).to(device)

objective = nn.CrossEntropyLoss()
optimizer = optim.Adam(params=net.parameters(), lr=0.01)
writer = SummaryWriter(comment="-frozenlake-search")

In [ ]:
'number of parameters', sum(p.numel() for p in net.parameters())

In [ ]:
full_batch = []

temperature = 3.0
gamma = 0.995

net.search.set_temperature(temperature)

with torch.device(device):
    for iter_no, batch in enumerate(iterate_batches(env, net, BATCH_SIZE)):
        
        reward_mean = float(np.mean(list(map(lambda s: s.reward, batch))))

        full_batch, obs, acts, reward_bound = filter_batch(full_batch + batch, PERCENTILE)

        if not full_batch:
            continue

        obs_v = torch.FloatTensor(obs).to(device)
        acts_v = torch.LongTensor(acts).to(device)
        full_batch = full_batch[-500:]

        optimizer.zero_grad()

        temperature = max(temperature * gamma, 1.0)

        net.search.set_temperature(temperature)

        action_scores_v = net(obs_v)

        loss_v = objective(action_scores_v, acts_v)

        loss_v.backward()
        optimizer.step()

        print("%d: loss=%.3f, rw_mean=%.3f, rw_bound=%.6f, batch=%d" % (
            iter_no, loss_v.item(), reward_mean, reward_bound, len(full_batch))
        )

        writer.add_scalar("loss", loss_v.item(), iter_no)
        writer.add_scalar("reward_mean", reward_mean, iter_no)
        writer.add_scalar("reward_bound", reward_bound, iter_no)

        if reward_mean > 0.8:
            print("Solved!")
            break

    writer.close()